*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-01-26 07:10:14--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  10.3MB/s    in 2.4s    

2022-01-26 07:10:17 (10.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [6]:
df_ratingCount = df_ratings.groupby(["isbn"]).count().reset_index()
often_rated_books = df_ratingCount.loc[df_ratingCount["rating"] >= 100]["isbn"]
ratingConditionSatisfied = df_books.loc[df_books["isbn"].isin(often_rated_books)]

dfUsers = df_ratings[["user", "rating"]].groupby(["user"]).count().reset_index()
userCondition = dfUsers.loc[dfUsers["rating"] >= 200]["user"]

df = df_ratings.loc[df_ratings["user"].isin(userCondition)]
df = df.loc[df["isbn"].isin(ratingConditionSatisfied["isbn"])]

df_pivot = df.pivot(index=["isbn"], columns=["user"], values=["rating"]).fillna(0)

In [7]:
df_pivot

rating                       ...                            
user       254    2276   2766   2977    ... 277427 277478 277639 278418
isbn                                    ...                            
002542730X    0.0    0.0    0.0    0.0  ...   10.0    0.0    0.0    0.0
0060008032    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
0060096195    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
006016848X    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
0060173289    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
...           ...    ...    ...    ...  ...    ...    ...    ...    ...
1573227331    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1573229326    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1573229571    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1592400876    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0
1878424319    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0

[727 rows x 888 columns]

In [8]:
matrix = df_pivot.values

In [9]:
neigh = NearestNeighbors(metric='cosine')
neigh.fit(matrix)


NearestNeighbors(metric='cosine')

In [10]:
# function to return recommended books - this will be tested
def get_recommends(book = ""): 
  
  try:
    bookFoundTrue = ratingConditionSatisfied.loc[ratingConditionSatisfied['title'] == book]
  except KeyError:
    print("Book not found")

  bookFoundTrue = ratingConditionSatisfied.loc[ratingConditionSatisfied['title'] == book]
  distance, indices = neigh.kneighbors(df_pivot.loc[df_pivot.index.isin(bookFoundTrue['isbn']),:].values.reshape(1,-1))

  recommended_books = [book]
  distance.flatten()

  results = []
  names = []
  counter = 0
  for i in indices[0]:
    _isbn = df_pivot.iloc[i].name
    title = df_books.loc[df_books['isbn'] == _isbn].values.flatten()
    name = title[1]
    names.append([name, distance[0][counter]])
    counter += 1
  names.pop(0)
  results.append(book)
  name = []
  [name.append(list(i)) for i in names[::-1]]
  results.append(name)
  recommended_books = results
  return recommended_books


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [13]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
# books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]
You passed the challenge! 🎉🎉🎉🎉🎉
